In [ ]:
!git clone https://github.com/Jaish19/Artificial-Intelligence.git

In [5]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.metrics import mean_absolute_error

# Disable some useless pandas warnings to prevent some junk from showing up in the output window
pd.options.mode.chained_assignment = None

# Load our data set
df = pd.read_csv("/content/Artificial-Intelligence/Neural Network Model/house_data.csv")

# Create the X and y arrays
X = df[["sq_feet", "num_bedrooms", "num_bathrooms"]]
y = df[["sale_price"]]

# Data needs to be scaled to  0 to 1 for the neural network to train correctly.
X_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X[X.columns] = X_scaler.fit_transform(X[X.columns])
y[y.columns] = y_scaler.fit_transform(y[y.columns])

# Split the data set in a training set (75%) and a test set (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Create a Neural Network model
model = Sequential()
model.add(Dense(50, input_dim=3, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(
    loss='mean_squared_error',
    optimizer='SGD'
)

# Train the model
model.fit(
    X_train,
    y_train,
    epochs=1,
    batch_size=8,
    shuffle=True,
    verbose=2
)

# Save the scalers to files so we can use it to pre-process new data later
joblib.dump(X_scaler, "/content/Artificial-Intelligence/Neural Network Model/X_scaler.pkl")
joblib.dump(y_scaler, "/content/Artificial-Intelligence/Neural Network Model/y_scaler.pkl")

# Save the trained model to a file so we can use it to make predictions later
model.save("/content/Artificial-Intelligence/Neural Network Model/house_value_model.h5")

# Report how well the model is performing
print("Model training results:")

# Report mean absolute error on the training set in a value scaled back to dollars so it's easier to understand.
predictions_train = model.predict(X_train, verbose=0)

mse_train = mean_absolute_error(
    y_scaler.inverse_transform(predictions_train),
    y_scaler.inverse_transform(y_train)
)
print(f" - Training Set Error: {mse_train}")

# Report mean absolute error on the test set in a value scaled back to dollars so it's easier to understand.
predictions_test = model.predict(X_test, verbose=0)

mse_test = mean_absolute_error(
    y_scaler.inverse_transform(predictions_test),
    y_scaler.inverse_transform(y_test)
)
print(f" - Test Set Error: {mse_test}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


938/938 - 2s - 3ms/step - loss: 0.0090


Model training results:
 - Training Set Error: 12459.7112125
 - Test Set Error: 12838.922359375001


#Test the trained model

In [7]:
from tensorflow import keras
# Load our trained model
model = keras.models.load_model('/content/Artificial-Intelligence/Neural Network Model/house_value_model.h5')

# Load the data scalers so that we can transform new data and prediction the same way as training data.
X_scaler = joblib.load('/content/Artificial-Intelligence/Neural Network Model/X_scaler.pkl')
y_scaler = joblib.load('/content/Artificial-Intelligence/Neural Network Model/y_scaler.pkl')

# Define the house that we want to value (with the values in the same order as in the training data).
house_1 = [
    2000,  # Size in Square Feet
    3,  # Number of Bedrooms
    2,  # Number of Bathrooms
]

# Keras assumes we want to predict the values for multiple of houses at once, so it expects an array.
# We only want to value a single house, so it will be the only item in our array.
homes = [
    house_1
]

# Scale the new data like the training data
scaled_home_data = X_scaler.transform(homes)

# Make a prediction for each house in the homes array (but we only have one)
home_values = model.predict(scaled_home_data)

# The prediction from the neural network will be scaled 0 to 1 just like the training data
# We need to unscale it using the same factor as we used to scale the training data
unscaled_home_values = y_scaler.inverse_transform(home_values)

# Since we are only predicting the price of one house, grab the first prediction returned
predicted_value = unscaled_home_values[0][0]

# Print the results
print("House details:")
print(f"- {house_1[0]} sq feet")
print(f"- {house_1[1]} bedrooms")
print(f"- {house_1[2]} bathrooms")
print(f"Estimated value: ${predicted_value:,.2f}")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step
House details:
- 2000 sq feet
- 3 bedrooms
- 2 bathrooms
Estimated value: $380,864.12
